### Fashion-MNIST

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_grad_enabled(True)

In [3]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()])
)

In [4]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)

### Network

In [12]:
from nn_fmnist import Network, get_num_correct

### Tensorboard

In [11]:
from torch.utils.tensorboard import SummaryWriter

comment = f' batch_size={batch_size} lr={lr}'
tb = SummaryWriter(comment=comment)
network = Network()

images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images, nrow=10)

tb.add_image('images', grid)
tb.add_graph(network, images)
tb.close()

### Hyperparameter tuning

In [15]:
from itertools import product

parameters = dict(
    lr = [.01, .001],
    batch_size = [10, 100, 1000],
    shuffle = [True, False]
)

In [16]:
param_values = [v for v in parameters.values()]
param_values

[[0.01, 0.001], [10, 100, 1000], [True, False]]

### Training

In [14]:
network = Network()
optimizer = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for batch in train_loader:

        images, labels = batch

        pred = network(images)
        loss = F.cross_entropy(pred, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += get_num_correct(pred, labels)
        
    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)
    
    tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
    tb.add_histogram('conv1.weight.grad', network.conv1.weight.grad, epoch)
    
    for name, wieght in network.named_parameters():
        
        tb.add_histogram(name, wieght, epoch)
        tb.add_histogram(f'{name}.grad', weight.grad, epoch)

    print("epoch:", epoch, "total correct: ", total_correct, "loss:", total_loss)

epoch: 0 total correct:  47595 loss: 328.7202114611864
epoch: 1 total correct:  51722 loss: 221.1665000319481
epoch: 2 total correct:  52409 loss: 205.01757775247097
epoch: 3 total correct:  52610 loss: 198.29786030948162
epoch: 4 total correct:  52789 loss: 194.3257301747799
epoch: 5 total correct:  52908 loss: 190.76685647666454
epoch: 6 total correct:  52884 loss: 189.94551321864128
epoch: 7 total correct:  53116 loss: 185.323159173131
epoch: 8 total correct:  53267 loss: 182.08751152455807
epoch: 9 total correct:  53521 loss: 177.1137904971838


In [ ]:
total_correct / len(train_set)

In [ ]:
prediction_loader = torch.utils.data.DataLoader(train_set, batch_size=10000)
train_preds = get_all_preds(network, prediction_loader)

In [ ]:
train_preds.shape

In [ ]:
print(train_preds.requires_grad)

In [ ]:
with torch.no_grad():
    
    prediction_loader = torch.utils.data.DataLoader(train_set, batch_size=10000)
    train_preds = get_all_preds(network, prediction_loader)

In [ ]:
print(train_preds.requires_grad)

In [ ]:
preds_correct = get_num_correct(train_preds, train_set.targets)

print('total correct:', preds_correct)
print('accuracy:', preds_correct / len(train_set))

### Confusion Matrix